<a href="https://colab.research.google.com/github/worldone-web/Team4-Traveler/blob/main/LightFM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808327 sha256=f1b52b5c7baabb1c543187d281aa8b936ee89fe8175256ecdc3789bf95fa6d9b
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [15]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k

In [4]:
df = pd.read_csv("/content/drive/MyDrive/G_Project/philadelphia_reviews.csv")
print(df.shape)
dataset = Dataset() #LightFm 데이터셋 생성
dataset.fit(users=df['user_id'], items=df['business_id']) # 데이터셋의 사용자 및 아이템 ID를 내부적으로 인덱싱

(738688, 3)


iterrows() 메서드를 통해 데이터프레임을 반복하며 각 행(row)에 접근하여 각 행마다 (index, row) 형태의 튜플로 반환한다.


---


각 행에 대해 사용자 ID(row['user_id']), 음식점 ID(row['business_id'])을
통해서 자동으로 상호작용값을 정한 뒤 모델을 생성


---


build_interactions() 메서드는 사용자-음식점 간의 상호 작용을 표현하는 희소 행렬을 반환한다.
희소 행렬은 상호 작용이 있는 부분에는 평점 값으로 채워지고, 상호 작용이 없는 부분은 0으로 채워진 행렬입니다.

---



In [11]:
interactions, _ = dataset.build_interactions((row['user_id'], row['business_id']) for index, row in df.iterrows())
train_data, test_data = random_train_test_split(interactions, test_percentage=0.2)

wrap은 LightFM모델에 사용되는 손실함수이며 사용자가 선호하는 음식점을 더 높은 순위로 배치하는 방식으로 동작한다.<br>
이 손실 함수는 추천 시스템의 핵심 지표 중 하나인 정확한 순위 매기기를 최적화하는데 유용하다.

---
여기서 test_data.row와 test_data.col은 테스트 데이터의 사용자와 음식점의 인덱스입니다. LightFM 모델은 사용자와 음식점 간의 상호 작용을 예측하기 위해 사용됩니다. 이 인덱스를 사용하여 각 사용자와 음식점 간의 상호 작용을 예측한 값이 predictions에 저장됩니다.


---

LightFM 모델의 predict 메서드는 사용자와 아이템의 인덱스를 입력으로 받으며<br>
예측값은 일반적으로 사용자와 음식점의 잠재적인 요인을 내적하여 얻어진다.


In [12]:
lightFm = LightFM(loss='warp')# , no_components=20)
lightFm.fit(train_data) # ,epochs =10)
# 예측 수행
predictions = lightFm.predict(test_data.row, test_data.col)

# RMSE 계산
rmse_score = np.sqrt(mean_squared_error(test_data.data, predictions))
print(f"RMSE: {rmse_score}")

# MAE 계산
mae_score = mean_absolute_error(test_data.data, predictions)
print(f"MAE: {mae_score}")

# MSE 계산
mse_score = mean_squared_error(test_data.data, predictions)
print(f"MSE: {mse_score}")


RMSE: 1.323287723373472
MAE: 1.0835964419418398
MSE: 1.751090398830946


해당 성능평가는 사용자의 예측 평점이 얼마나 정확한지로 평가했다.

LightFM은 평가 지표를 직접 계산하는 기능을 제공하지 않는다. 따라서 위의 코드에서 사용한 RMSE(Root Mean Squared Error), MAE(Mean Absolute Error), MSE(Mean Squared Error)는 LightFM에서 제공하는 메서드가 아니며, 올바른 평가 방법이 아니다.

AUC는 모델이 랭킹을 얼마나 잘 예측하는지 측정하며,<br> precision at k는 상위 k개 아이템 중 실제로 사용자가 선호하는 아이템의 비율을 측정한다.

In [16]:
# AUC 평가
train_auc = auc_score(lightFm, train_data).mean()
test_auc = auc_score(lightFm, test_data).mean()

# Precision at k 평가
k = 5  # 상위 k개 아이템
train_precision = precision_at_k(lightFm, train_data, k=k).mean()
test_precision = precision_at_k(lightFm, test_data, k=k).mean()

print("Train AUC:", train_auc)
print("Test AUC:", test_auc)
print("Train Precision at", k, ":", train_precision)
print("Test Precision at", k, ":", test_precision)

Train AUC: 0.84907186
Test AUC: 0.83860433
Train Precision at 5 : 0.018789612
Test Precision at 5 : 0.011333432


In [18]:
# Precision at k 평가
k = 1  # 상위 k개 아이템
train_precision = precision_at_k(lightFm, train_data, k=k).mean()
test_precision = precision_at_k(lightFm, test_data, k=k).mean()
print("Train Precision at", k, ":", train_precision)
print("Test Precision at", k, ":", test_precision)

Train Precision at 1 : 0.025683071
Test Precision at 1 : 0.015481233


Precision at k는 상위 k개 아이템에 대한 정확도를 나타내는 지표이다. Precision at k는 추천된 상위 k개 아이템 중에서 사용자가 실제로 선호하는 아이템의 비율을 측정하여 사용자에게 얼마나 정확한 추천을 제공하는지를 평가하는 데 사용된다. Precision at k가 높을수록 모델이 더 정확한 추천을 제공한다는 의미
1에 가까울수록 좋음

AUC는 모델이 얼마나 잘 양성 클래스와 음성 클래스를 분리하는지를 나타낸다. LightFM에서는 이진 분류 문제가 아닌 협업 필터링(추천) 문제를 다루기 때문에 AUC는 사용자에게 올바른 아이템을 추천하는 능력을 측정한다. AUC가 1에 가까울수록 모델이 더 좋은 추천을 제공한다는 의미

In [17]:
# AUC 평가
train_auc = auc_score(lightFm, train_data)
test_auc = auc_score(lightFm, test_data)

# Precision at k 평가
k = 5  # 상위 k개 아이템
train_precision = precision_at_k(lightFm, train_data, k=k)
test_precision = precision_at_k(lightFm, test_data, k=k)

print("Train AUC:", train_auc)
print("Test AUC:", test_auc)
print("Train Precision at", k, ":", train_precision)
print("Test Precision at", k, ":", test_precision)

Train AUC: [0.9137809  0.7729882  0.8032335  ... 0.38897526 0.37017667 0.17484099]
Test AUC: [0.82611    0.8720659  0.86982334 ... 0.62233216 0.15505302 0.08734983]
Train Precision at 5 : [0. 0. 0. ... 0. 0. 0.]
Test Precision at 5 : [0. 0. 0. ... 0. 0. 0.]


위는 각 사용자에 대한 모델의 성능이다.